# <center> Learned primal-dual</center>

Import libraries.

In [1]:
import os
import adler
adler.util.gpu.setup_one_gpu(0)

Picking GPU 0


In [2]:
from adler.odl.phantom import random_phantom
from adler.tensorflow import prelu, cosine_decay

In [3]:
import tensorflow as tf
import numpy as np
import odl
import odl.contrib.tensorflow

In [4]:
np.random.seed(0)
#name = os.path.splitext(os.path.basename(__file__))[0]
name = os.path.splitext(os.getcwd())[0]+'/checkpoints'

In [6]:
# Creating checkpoints directory
try:
    os.mkdir(name)
except OSError:
    print ("Creation of the directory %s failed" % name)
else:
    print ("Successfully created the directory %s " % name)

Successfully created the directory /store/kepler/datastore/andrade/GitHub_repos/Joint_CTWF_Recon/Microcanonical_NN/checkpoints 


In [5]:
sess = tf.InteractiveSession()

Define space and geometry

In [7]:
# Create ODL data structures
size = 128
space = odl.uniform_discr([-64, -64], [64, 64], [size, size],
                          dtype='float32')

In [8]:
geometry = odl.tomo.parallel_beam_geometry(space, num_angles=30)

In [9]:
operator = odl.tomo.RayTransform(space, geometry)

In [10]:
# Ensure operator has fixed operator norm for scale invariance
opnorm = odl.power_method_opnorm(operator)
operator = (1 / opnorm) * operator

In [11]:
operator.operator

RayTransform: uniform_discr([-64., -64.], [ 64.,  64.], (128, 128), dtype='float32') -> uniform_discr([  0.    , -90.5097], [  3.1416,  90.5097], (30, 183), dtype='float32', weighting=1.0)

In [12]:
# Create tensorflow layer from odl operator
odl_op_layer = odl.contrib.tensorflow.as_tensorflow_layer(operator,
                                                          'RayTransform')
odl_op_layer_adjoint = odl.contrib.tensorflow.as_tensorflow_layer(operator.adjoint,
                                                                  'RayTransformAdjoint')

In [13]:
# User selected paramters
n_data = 5
n_iter = 10
n_primal = 5
n_dual = 5

In [14]:
def generate_data(validation=False):
    """Generate a set of random data."""
    n_generate = 1 if validation else n_data

    y_arr = np.empty((n_generate, operator.range.shape[0], operator.range.shape[1], 1), dtype='float32')
    x_true_arr = np.empty((n_generate, space.shape[0], space.shape[1], 1), dtype='float32')

    for i in range(n_generate):
        if validation:
            phantom = odl.phantom.shepp_logan(space, True)
        else:
            phantom = random_phantom(space)
        data = operator(phantom)
        noisy_data = data + odl.phantom.white_noise(operator.range) * np.mean(np.abs(data)) * 0.05

        x_true_arr[i, ..., 0] = phantom
        y_arr[i, ..., 0] = noisy_data

    return y_arr, x_true_arr

In [15]:
with tf.name_scope('placeholders'):
    x_true = tf.placeholder(tf.float32, shape=[None, size, size, 1], name="x_true")
    y_rt = tf.placeholder(tf.float32, shape=[None, operator.range.shape[0], operator.range.shape[1], 1], name="y_rt")
    is_training = tf.placeholder(tf.bool, shape=(), name='is_training')

In [16]:
x_true

<tf.Tensor 'placeholders/x_true:0' shape=(?, 128, 128, 1) dtype=float32>

In [17]:
y_arr, x_true_arr = generate_data()

In [18]:
x_true

<tf.Tensor 'placeholders/x_true:0' shape=(?, 128, 128, 1) dtype=float32>

In [19]:
x_true_arr.shape

(5, 128, 128, 1)

In [20]:
def apply_conv(x, filters=32):
    return tf.layers.conv2d(x, filters=filters, kernel_size=3, padding='SAME',
                            kernel_initializer=tf.contrib.layers.xavier_initializer())

In [25]:
with tf.name_scope('tomography'):
    with tf.name_scope('initial_values'):
        primal = tf.concat([tf.zeros_like(x_true)] * n_primal, axis=-1)
        dual = tf.concat([tf.zeros_like(y_rt)] * n_dual, axis=-1)

    for i in range(n_iter):
        with tf.variable_scope('dual_iterate_{}'.format(i)):
            evalop = odl_op_layer(primal[..., 1:2])
            update = tf.concat([dual, evalop, y_rt], axis=-1)

            #update = prelu(apply_conv(update), name='prelu_1')
            #update = prelu(apply_conv(update), name='prelu_2')
            update = apply_conv(update)
            update = apply_conv(update)
            update = apply_conv(update, filters=n_dual)
            dual = dual + update

        with tf.variable_scope('primal_iterate_{}'.format(i)):
            evalop = odl_op_layer_adjoint(dual[..., 0:1])
            update = tf.concat([primal, evalop], axis=-1)

            #update = prelu(apply_conv(update), name='prelu_1')
            #update = prelu(apply_conv(update), name='prelu_2')
            update = apply_conv(update)
            update = apply_conv(update)
            update = apply_conv(update, filters=n_primal)
            primal = primal + update

    x_result = primal[..., 0:1]

In [26]:
x_result

<tf.Tensor 'tomography_1/strided_slice:0' shape=(?, 128, 128, 1) dtype=float32>

In [35]:
name

'/store/kepler/datastore/andrade/GitHub_repos/Joint_CTWF_Recon/Microcanonical_NN/checkpoints'

In [27]:
with tf.name_scope('loss'):
    residual = x_result - x_true
    squared_error = residual ** 2
    loss = tf.reduce_mean(squared_error)

In [28]:
with tf.name_scope('optimizer'):
    # Learning rate
    global_step = tf.Variable(0, trainable=False)
    maximum_steps = 100001
    starter_learning_rate = 1e-3
    learning_rate = cosine_decay(starter_learning_rate,
                                 global_step,
                                 maximum_steps,
                                 name='learning_rate')

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt_func = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                          beta2=0.99)

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), 1)
        optimizer = opt_func.apply_gradients(zip(grads, tvars),
                                             global_step=global_step)

In [36]:
ckp_name = name+'_lpd_wavelets/checkpoints'

In [38]:
# Summaries
# tensorboard --logdir=...

with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('psnr', -10 * tf.log(loss) / tf.log(10.0))

    tf.summary.image('x_result', x_result)
    tf.summary.image('x_true', x_true)
    tf.summary.image('squared_error', squared_error)
    tf.summary.image('residual', residual)

    merged_summary = tf.summary.merge_all()
    test_summary_writer = tf.summary.FileWriter(adler.tensorflow.util.default_tensorboard_dir(ckp_name) + '/test',
                                                sess.graph)
    train_summary_writer = tf.summary.FileWriter(adler.tensorflow.util.default_tensorboard_dir(ckp_name) + '/train')

In [39]:
# Initialize all TF variables
sess.run(tf.global_variables_initializer())

In [40]:
# Add op to save and restore
saver = tf.train.Saver()

In [41]:
# Generate validation data
y_arr_validate, x_true_arr_validate = generate_data(validation=True)

In [43]:
if 0:
    saver.restore(sess,
                  adler.tensorflow.util.default_checkpoint_path(ckp_name))

Train the network

In [44]:
# Train the network
for i in range(0, maximum_steps):
    if i%10 == 0:
        y_arr, x_true_arr = generate_data()

    _, merged_summary_result_train, global_step_result = sess.run([optimizer, merged_summary, global_step],
                              feed_dict={x_true: x_true_arr,
                                         y_rt: y_arr,
                                         is_training: True})

    if i>0 and i%10 == 0:
        loss_result, merged_summary_result, global_step_result = sess.run([loss, merged_summary, global_step],
                              feed_dict={x_true: x_true_arr_validate,
                                         y_rt: y_arr_validate,
                                         is_training: False})

        train_summary_writer.add_summary(merged_summary_result_train, global_step_result)
        test_summary_writer.add_summary(merged_summary_result, global_step_result)

        print('iter={}, loss={}'.format(global_step_result, loss_result))

    if i>0 and i%1000 == 0:
        saver.save(sess,
                   adler.tensorflow.util.default_checkpoint_path(ckp_name))

iter=11, loss=0.29636240005493164
iter=21, loss=0.0379224494099617
iter=31, loss=0.0366794615983963
iter=41, loss=0.0287071093916893
iter=51, loss=0.025743922218680382
iter=61, loss=0.021135233342647552
iter=71, loss=0.019079813733696938
iter=81, loss=0.018233753740787506
iter=91, loss=0.01719268597662449
iter=101, loss=0.01605924591422081
iter=111, loss=0.01504405029118061
iter=121, loss=0.014368291944265366
iter=131, loss=0.013987390324473381
iter=141, loss=0.013721946626901627
iter=151, loss=0.013516995124518871
iter=161, loss=0.01346299797296524
iter=171, loss=0.013354815542697906
iter=181, loss=0.012837867252528667
iter=191, loss=0.012525378726422787
iter=201, loss=0.012221596203744411
iter=211, loss=0.012356549501419067
iter=221, loss=0.025662817060947418
iter=231, loss=0.022136015817523003
iter=241, loss=0.016285210847854614
iter=251, loss=0.015316985547542572
iter=261, loss=0.014464205130934715
iter=271, loss=0.012912250123918056
iter=281, loss=0.012608591467142105
iter=291, lo

iter=2271, loss=0.010221108794212341
iter=2281, loss=0.011239899322390556
iter=2291, loss=0.011864576488733292
iter=2301, loss=0.01446986198425293
iter=2311, loss=0.012088684365153313
iter=2321, loss=0.012198498472571373
iter=2331, loss=0.010433917865157127
iter=2341, loss=0.01018168032169342
iter=2351, loss=0.009972902946174145
iter=2361, loss=0.00994805060327053
iter=2371, loss=0.010554981417953968
iter=2381, loss=0.010020613670349121
iter=2391, loss=0.009683351963758469
iter=2401, loss=0.009675438515841961
iter=2411, loss=0.0096256323158741
iter=2421, loss=0.009609240107238293
iter=2431, loss=0.009781657718122005
iter=2441, loss=0.010348495095968246
iter=2451, loss=0.010026061907410622
iter=2461, loss=0.009718598797917366
iter=2471, loss=0.009730430319905281
iter=2481, loss=0.009659264236688614
iter=2491, loss=0.009764319285750389
iter=2501, loss=0.009603156708180904
iter=2511, loss=0.009613748639822006
iter=2521, loss=0.009635180234909058
iter=2531, loss=0.00955907627940178
iter=25

iter=4501, loss=0.009621663019061089
iter=4511, loss=0.009953988716006279
iter=4521, loss=0.010180002078413963
iter=4531, loss=0.009731605648994446
iter=4541, loss=0.009486427530646324
iter=4551, loss=0.009564196690917015
iter=4561, loss=0.00966242142021656
iter=4571, loss=0.009805304929614067
iter=4581, loss=0.00959749985486269
iter=4591, loss=0.00953901931643486
iter=4601, loss=0.009528232738375664
iter=4611, loss=0.0098344124853611
iter=4621, loss=0.009834785014390945
iter=4631, loss=0.010603762231767178
iter=4641, loss=0.009862883947789669
iter=4651, loss=0.009615936316549778
iter=4661, loss=0.01001846045255661
iter=4671, loss=0.009769700467586517
iter=4681, loss=0.009847482666373253
iter=4691, loss=0.010066721588373184
iter=4701, loss=0.009785670787096024
iter=4711, loss=0.009526606649160385
iter=4721, loss=0.009518135339021683
iter=4731, loss=0.009484682232141495
iter=4741, loss=0.009756684303283691
iter=4751, loss=0.009656737558543682
iter=4761, loss=0.009557252749800682
iter=47

iter=6731, loss=0.009609822183847427
iter=6741, loss=0.010301806032657623
iter=6751, loss=0.009894819930195808
iter=6761, loss=0.009779822081327438
iter=6771, loss=0.009928321465849876
iter=6781, loss=0.009713840670883656
iter=6791, loss=0.009960079565644264
iter=6801, loss=0.009760476648807526
iter=6811, loss=0.009665369987487793
iter=6821, loss=0.009966226294636726
iter=6831, loss=0.009684409946203232
iter=6841, loss=0.009628832340240479
iter=6851, loss=0.009616821072995663
iter=6861, loss=0.009663489647209644
iter=6871, loss=0.009595214389264584
iter=6881, loss=0.00947156548500061
iter=6891, loss=0.009672741405665874
iter=6901, loss=0.009634916670620441
iter=6911, loss=0.009578953497111797
iter=6921, loss=0.009656109847128391
iter=6931, loss=0.009831996634602547
iter=6941, loss=0.009739505127072334
iter=6951, loss=0.009749533608555794
iter=6961, loss=0.009822418913245201
iter=6971, loss=0.010237529873847961
iter=6981, loss=0.0099721048027277
iter=6991, loss=0.009638679213821888
iter

iter=8961, loss=0.011544758453965187
iter=8971, loss=0.010180201381444931
iter=8981, loss=0.01005655899643898
iter=8991, loss=0.009798136539757252
iter=9001, loss=0.009620435535907745
iter=9011, loss=0.00974910520017147
iter=9021, loss=0.00977979227900505
iter=9031, loss=0.009629960171878338
iter=9041, loss=0.010179618373513222
iter=9051, loss=0.01011630892753601
iter=9061, loss=0.009832445532083511
iter=9071, loss=0.009673934429883957
iter=9081, loss=0.009622987359762192
iter=9091, loss=0.009622832760214806
iter=9101, loss=0.009677301160991192
iter=9111, loss=0.009570546448230743
iter=9121, loss=0.009538653306663036
iter=9131, loss=0.009518701583147049
iter=9141, loss=0.009581159800291061
iter=9151, loss=0.009632216766476631
iter=9161, loss=0.009553181007504463
iter=9171, loss=0.009896277450025082
iter=9181, loss=0.009637880139052868
iter=9191, loss=0.009548548609018326
iter=9201, loss=0.010034821927547455
iter=9211, loss=0.009713295847177505
iter=9221, loss=0.009587051346898079
iter=

...

iter=89291, loss=0.009459629654884338
iter=89301, loss=0.009448475204408169
iter=89311, loss=0.009432101622223854
iter=89321, loss=0.009418257512152195
iter=89331, loss=0.009407865814864635
iter=89341, loss=0.009398926049470901
iter=89351, loss=0.00943021010607481
iter=89361, loss=0.009457988664507866
iter=89371, loss=0.009462191723287106
iter=89381, loss=0.009460117667913437
iter=89391, loss=0.009443433955311775
iter=89401, loss=0.00942349061369896
iter=89411, loss=0.009410993196070194
iter=89421, loss=0.009405635297298431
iter=89431, loss=0.009413912892341614
iter=89441, loss=0.00942572858184576
iter=89451, loss=0.009432146325707436
iter=89461, loss=0.00942267570644617
iter=89471, loss=0.009441985748708248
iter=89481, loss=0.009449532255530357
iter=89491, loss=0.009420568123459816
iter=89501, loss=0.009413275867700577
iter=89511, loss=0.00941132940351963
iter=89521, loss=0.009397042915225029
iter=89531, loss=0.009395379573106766
iter=89541, loss=0.009386120364069939
iter=89551, loss=

iter=91461, loss=0.009286683052778244
iter=91471, loss=0.009294692426919937
iter=91481, loss=0.009297912940382957
iter=91491, loss=0.009290926158428192
iter=91501, loss=0.009280356578528881
iter=91511, loss=0.009278200566768646
iter=91521, loss=0.009280461817979813
iter=91531, loss=0.009279384277760983
iter=91541, loss=0.009284546598792076
iter=91551, loss=0.009283682331442833
iter=91561, loss=0.009280501864850521
iter=91571, loss=0.009270742535591125
iter=91581, loss=0.00927101168781519
iter=91591, loss=0.009281933307647705
iter=91601, loss=0.00927187129855156
iter=91611, loss=0.009263097308576107
iter=91621, loss=0.009255897253751755
iter=91631, loss=0.009245332330465317
iter=91641, loss=0.009254524484276772
iter=91651, loss=0.009259832091629505
iter=91661, loss=0.009270275942981243
iter=91671, loss=0.009279602207243443
iter=91681, loss=0.009288636036217213
iter=91691, loss=0.009294581599533558
iter=91701, loss=0.009292999282479286
iter=91711, loss=0.009294797666370869
iter=91721, lo

iter=93631, loss=0.009387396275997162
iter=93641, loss=0.009390726685523987
iter=93651, loss=0.009384054690599442
iter=93661, loss=0.009379297494888306
iter=93671, loss=0.009379848837852478
iter=93681, loss=0.009376049041748047
iter=93691, loss=0.009370507672429085
iter=93701, loss=0.009364522062242031
iter=93711, loss=0.00935879535973072
iter=93721, loss=0.009349262341856956
iter=93731, loss=0.009347723796963692
iter=93741, loss=0.009347155690193176
iter=93751, loss=0.009343355894088745
iter=93761, loss=0.009340692311525345
iter=93771, loss=0.00934139546006918
iter=93781, loss=0.009337146766483784
iter=93791, loss=0.009326470084488392
iter=93801, loss=0.0093205776065588
iter=93811, loss=0.009318819269537926
iter=93821, loss=0.009317799471318722
iter=93831, loss=0.0093148872256279
iter=93841, loss=0.00930829532444477
iter=93851, loss=0.00930219516158104
iter=93861, loss=0.009301219135522842
iter=93871, loss=0.009302200749516487
iter=93881, loss=0.009300531819462776
iter=93891, loss=0.0

iter=95801, loss=0.009347427636384964
iter=95811, loss=0.009347440674901009
iter=95821, loss=0.009346520528197289
iter=95831, loss=0.009343819692730904
iter=95841, loss=0.009344739839434624
iter=95851, loss=0.009343845769762993
iter=95861, loss=0.009345579892396927
iter=95871, loss=0.009347430430352688
iter=95881, loss=0.009347138926386833
iter=95891, loss=0.009342731907963753
iter=95901, loss=0.009339291602373123
iter=95911, loss=0.00933690182864666
iter=95921, loss=0.009335866197943687
iter=95931, loss=0.00933302566409111
iter=95941, loss=0.009329277090728283
iter=95951, loss=0.009327463805675507
iter=95961, loss=0.009326462633907795
iter=95971, loss=0.009325796738266945
iter=95981, loss=0.00932391732931137
iter=95991, loss=0.009324326179921627
iter=96001, loss=0.009323288686573505
iter=96011, loss=0.00932214967906475
iter=96021, loss=0.0093223936855793
iter=96031, loss=0.009321983903646469
iter=96041, loss=0.009331987239420414
iter=96051, loss=0.009336788207292557
iter=96061, loss=0

iter=97971, loss=0.00931309163570404
iter=97981, loss=0.009313028305768967
iter=97991, loss=0.009312883950769901
iter=98001, loss=0.009312704205513
iter=98011, loss=0.009313005954027176
iter=98021, loss=0.009313493967056274
iter=98031, loss=0.009313871152698994
iter=98041, loss=0.009315351955592632
iter=98051, loss=0.009316532872617245
iter=98061, loss=0.009316619485616684
iter=98071, loss=0.009316133335232735
iter=98081, loss=0.009316349402070045
iter=98091, loss=0.009315983392298222
iter=98101, loss=0.009316516108810902
iter=98111, loss=0.009317072108387947
iter=98121, loss=0.009316965937614441
iter=98131, loss=0.0093163400888443
iter=98141, loss=0.009315446019172668
iter=98151, loss=0.009314551018178463
iter=98161, loss=0.009314032271504402
iter=98171, loss=0.009313954040408134
iter=98181, loss=0.009313870221376419
iter=98191, loss=0.009313542395830154
iter=98201, loss=0.009312722831964493
iter=98211, loss=0.009312540292739868
iter=98221, loss=0.009312305599451065
iter=98231, loss=0